- what is the difference between adding more layers or filters?
- what is the impact of changing each of the params values?
- what is an epoch?


In [1]:
import os
from tensorflow.keras import models, layers, activations, optimizers, utils, losses, initializers, metrics, callbacks 

In [21]:
path = 'train'
epochs = 100
batch_size = 32
patience = 12
learning_rate = 0.001 # alpha from sgd
model_path = 'model.keras' # fro dump

In [ ]:


model = models.Sequential([             # all layers connected - most simple one - used 99% of the time
                           
    layers.Resizing(56, 56),            # used to lower params by image -- may cause problems due to resizing as images do not lose data but computer may not understand different ratios
    layers.Rescaling(1.0/255),          # values goes from 0-255 to 0.0-1.0 
    layers.RandomRotation((-0.2, 0.2)),
    
    layers.Conv2D(32, (3, 3),           # 32 @ 54x54
                  activation='relu',
                  kernel_initializer=initializers.RandomNormal()
    ),
    
    layers.MaxPooling2D((2, 2)),         # 32 @ 27x27
    
    layers.Conv2D(64, (3, 3),           # 32 @ 54x54
                  activation='relu',
                  kernel_initializer=initializers.RandomNormal()
    ),
    
    layers.MaxPooling2D((2, 2)),         # 2048 @ 8x8
    
    layers.Flatten(),                   # ~125k params
    
    layers.Dropout(0.2),
    
    layers.Dense(128,
                 activation='relu',
                 kernel_initializer=initializers.RandomNormal()
    ),
    
    layers.Dense(64,
                 activation='relu',
                 kernel_initializer=initializers.RandomNormal()
    ),
        
    layers.Dense(64,
                 activation='relu',
                 kernel_initializer=initializers.RandomNormal()
    ),
    
    layers.Dense(1,
                 activations='sigmoid',
                 kernel_initializer=initializers.RandomNormal()
    ),
]) 



In [9]:
model.compile(
    optimizer=optimizers.Adam(
        learning_rate=learning_rate
    ),
    loss=losses.BinaryCrossentropy(),
    metrics=[metrics.BinaryAccuracy(), metrics.Precision(), metrics.Recall()]
)

In [13]:
train = utils.image_dataset_from_directory(
    path,
    validation_split=0.2,
    subset="training",
    shuffle=True,
    seed=811,
    image_size=(224, 224),
    batch_size = batch_size
)

Found 11879 files belonging to 2 classes.
Using 9504 files for training.


In [15]:
test = utils.image_dataset_from_directory(
    path,
    validation_split=0.2,
    subset="validation",
    shuffle=True,
    seed=811,
    image_size=(224, 224),
    batch_size = batch_size
)

Found 13879 files belonging to 2 classes.
Using 2775 files for validation.


In [22]:
model.fit(
    train, 
    validation_data=test,
    epochs=epochs,
    callbacks = [
        callbacks.EarlyStopping(
            monitor='val_loss',
            patience=patience,
            verbose=1
        ),
        callbacks.ModelCheckpoint(
            filepath = model_path,
            save_weights_only=False,
            monitor="loss",
            mode="min",
            save_best_only=True
        )
    ]
)

Epoch 1/100
297/297 ━━━━━━━━━━━━━━━━━━━━ 8s 27ms/step - binary_accuracy: 0.9315 - loss: 0.1675 - precision: 0.9344 - recall: 0.9197 - val_binary_accuracy: 0.9117 - val_loss: 0.2472 - val_precision: 0.8873 - val_recall: 0.9379
Epoch 2/100
297/297 ━━━━━━━━━━━━━━━━━━━━ 8s 26ms/step - binary_accuracy: 0.9264 - loss: 0.1755 - precision: 0.9264 - recall: 0.9165 - val_binary_accuracy: 0.9070 - val_loss: 0.2374 - val_precision: 0.8935 - val_recall: 0.9186
Epoch 3/100
297/297 ━━━━━━━━━━━━━━━━━━━━ 8s 27ms/step - binary_accuracy: 0.9303 - loss: 0.1668 - precision: 0.9311 - recall: 0.9203 - val_binary_accuracy: 0.9099 - val_loss: 0.2477 - val_precision: 0.8970 - val_recall: 0.9209
Epoch 4/100
297/297 ━━━━━━━━━━━━━━━━━━━━ 8s 27ms/step - binary_accuracy: 0.9282 - loss: 0.1758 - precision: 0.9345 - recall: 0.9116 - val_binary_accuracy: 0.8933 - val_loss: 0.2517 - val_precision: 0.9151 - val_recall: 0.8609
Epoch 5/100
297/297 ━━━━━━━━━━━━━━━━━━━━ 8s 27ms/step - binary_accuracy: 0.9292 - loss: 0.1724 -